# Canarium GBS Assembly
### *Federman et al.*

This notebook provides all code necessary to reproduce the assembled GBS data sets used in Federman et al. (xxxx). Starting from demultiplexed fastq data files we assemble the data into four complete data sets that were used in downstream analyses. All code in this notebook is written in Python and uses the *ipyrad* package for assembly. 

### Required software

In [6]:
## conda install ipyrad -c ipyrad
## conda install sra-tools -c bioconda
## conda install entrez-direct -c bioconda

### Imports

In [7]:
import ipyrad as ip
import ipyrad.analysis as ipa

In [8]:
print "ipyrad v.{}".format(ip.__version__)

ipyrad v.0.7.20


### Connect to cluster

In [9]:
import ipyparallel as ipp
ipyclient = ipp.Client()
ip.cluster_info(ipyclient)

host compute node: [40 cores] on sacra


## Links to Sections

+ [Download Sequence Data](#Download-sequence-data-from-SRA)
+ [ipyrad Assembly](#ipyrad-Assembly)
+ [Assembly stats](#Assembly-stats)

### Download sequence data from SRA

In [10]:
## get accession data from sra
sra = ipa.sratools(accession="SRP106882", workdir="./fastq-files")

## print run info for posterity
run_info = sra.fetch_runinfo((1,4,6,29,30))
run_info

Fetching project data...

,Run,spots,spots_with_mates,ScientificName,SampleName
0,SRR5534921,12818693,0,Canarium lamianum,SF327
1,SRR5534922,5675773,0,Canarium longistipulatum,D12950
2,SRR5534923,34404746,0,Canarium ovatum,D14269
3,SRR5534924,3382649,0,Canarium pilicarpum,5573
4,SRR5534925,16632442,0,Canarium obtusifolium,SF228
5,SRR5534926,10769159,0,Canarium odontophyllum,SFC1988
6,SRR5534927,47625,0,Canarium ntidifolium,4304
7,SRR5534928,17034881,0,Canarium obtusifolium,SF224
8,SRR5534929,211932,0,Canarium ferrugineum,SF343
9,SRR5534930,402493,0,Canarium galokense,D13101


In [4]:
## run parallel download
sra.run(ipyclient=ipyclient)

[####################] 100%  Downloading fastq files | 0:24:21 |  
48 fastq files downloaded to /home/deren/Documents/Canarium/fastq-files


## ipyrad Assembly

Enter parameter values for the ipyrad assembly . 

In [7]:
## create an Assembly
data = ip.Assembly("Canarium")

## set params
data.set_params("project_dir", "analysis-ipyrad")
data.set_params("sorted_fastq_path", "./fastq-files/*.gz")
data.set_params("restriction_overhang", ("CWGC", "CWGC"))
data.set_params("datatype", "gbs")
data.set_params("clust_threshold", 0.90)
data.set_params("filter_adapters", 2)
data.set_params("max_SNPs_locus", (10, 10))
data.set_params("max_shared_Hs_locus", 4)
data.set_params("trim_reads", (0, 0))
data.set_params("trim_loci", (0, 5))
data.set_params("output_formats", list("lpsvka"))

## print params for posterity
data.get_params()

New Assembly: Canarium
0   assembly_name               Canarium                                     
1   project_dir                 ./analysis-ipyrad                            
2   raw_fastq_path                                                           
3   barcodes_path                                                            
4   sorted_fastq_path           ./fastq-files/*.gz                           
5   assembly_method             denovo                                       
6   reference_sequence                                                       
7   datatype                    gbs                                          
8   restriction_overhang        ('CWGC', 'CWGC')                             
9   max_low_qual_bases          5                                            
10  phred_Qscore_offset         33                                           
11  mindepth_statistical        6                                            
12  mindepth_majrule            6        

### Assemble reads within each Sample

In [ ]:
data.run("12")

In [5]:
data.run("3", force=True)

Assembly: Canarium
[####################] 100%  dereplicating         | 0:01:55 | s3 | | 
[####################] 100%  clustering            | 15:30:31 | s3 | 
[####################] 100%  building clusters     | 0:04:02 | s3 | 
[####################] 100%  chunking              | 0:00:42 | s3 | 
[####################] 100%  aligning              | 1:00:07 | s3 | 
[####################] 100%  concatenating         | 0:03:13 | s3 | 


In [6]:
data.run("45")

Assembly: Canarium
[####################] 100%  inferring [H, E]      | 0:12:06 | s4 | 
[####################] 100%  calculating depths    | 0:00:54 | s5 | 
[####################] 100%  chunking clusters     | 0:01:22 | s5 | 
[####################] 100%  consens calling       | 0:21:34 | s5 | 


In [7]:
data.run("6")

Assembly: Canarium
[####################] 100%  concat/shuffle input  | 0:01:06 | s6 | 
[####################] 100%  clustering across     | 5:43:06 | s6 | 
[####################] 100%  building clusters     | 0:00:53 | s6 | 
[####################] 100%  aligning clusters     | 0:04:13 | s6 | 
[####################] 100%  database indels       | 0:02:13 | s6 | 
[####################] 100%  indexing clusters     | 0:02:19 | s6 | 
[####################] 100%  building database     | 0:24:48 | s6 | 


In [8]:
data.run("7")

Assembly: Canarium
[####################] 100%  filtering loci        | 0:00:39 | s7 | 
[####################] 100%  building loci/stats   | 0:00:29 | s7 | 
[####################] 100%  building alleles      | 0:00:36 | s7 | 
[####################] 100%  building vcf file     | 0:01:06 | s7 | 
[####################] 100%  writing vcf file      | 0:00:00 | s7 | 
[####################] 100%  building arrays       | 0:00:31 | s7 | 
[####################] 100%  writing outfiles      | 0:01:00 | s7 | 
Outfiles written to: ~/Documents/Canarium/analysis-ipyrad/Canarium_outfiles



### Full assembly stats

In [8]:
## re-load assembly in case coming back to this notebook later
data = ip.load_json("analysis-ipyrad/Canarium.json")

loading Assembly: Canarium
from saved path: ~/Documents/Canarium/analysis-ipyrad/Canarium.json


In [10]:
## print some stats
print "total N reads:", data.stats.reads_raw.sum()
print "mean N reads/sample:", data.stats.reads_raw.mean()
print "S.D. N reads/sample:", data.stats.reads_raw.std()


total N reads: 365012834
mean N reads/sample: 7604434.04167
S.D. N reads/sample: 7407655.17926


### Excluding low-data samples

In [9]:
## number of consens reads per sample. 
data.stats_dfs.s7_samples

,sample_coverage
4304,7
5573,37672
D12950,44861
D12962,1981
D12963,10141
D13052,75316
D13053,21812
D13063,28227
D13075,4096
D13090,27541


In [47]:
## make subset lists to exclude taxa with little data
subs = [i.name for i in data.samples.values() if i.stats.reads_consens > 12000]
subsnout = list(set(subs) - set(["D14269", "D13374", "SFC1988", "D13852"]))

### Final assemblies

In [48]:
## make new branches
Can_min4 = data.branch("Canarium-min4", subsamples=subs)
Can_min10 = data.branch("Canarium-min10", subsamples=subs)
Can_min20 = data.branch("Canarium-min20", subsamples=subs)
Can_min30nout = data.branch("Canarium-min30-nout", subsamples=subsnout)

## set params on new assemblies
Can_min10.set_params("min_samples_locus", 10)
Can_min20.set_params("min_samples_locus", 20)
Can_min30nout.set_params("min_samples_locus", 30)

In [49]:
## final assemblies
Can_min4.run("7", force=True)
Can_min10.run("7", force=True)
Can_min20.run("7", force=True)
Can_min30nout.run("7", force=True)

Assembly: Canarium-min4
[####################] 100%  filtering loci        | 0:00:50 | s7 | 
[####################] 100%  building loci/stats   | 0:00:29 | s7 | 
[####################] 100%  building alleles      | 0:00:36 | s7 | 
[####################] 100%  building vcf file     | 0:01:00 | s7 | 
[####################] 100%  writing vcf file      | 0:00:00 | s7 | 
[####################] 100%  building arrays       | 0:00:40 | s7 | 
[####################] 100%  writing outfiles      | 0:00:50 | s7 | 
Outfiles written to: ~/Documents/Canarium/analysis-ipyrad/Canarium-min4_outfiles

Assembly: Canarium-min10
[####################] 100%  filtering loci        | 0:00:41 | s7 | 
[####################] 100%  building loci/stats   | 0:00:29 | s7 | 
[####################] 100%  building alleles      | 0:00:35 | s7 | 
[####################] 100%  building vcf file     | 0:00:49 | s7 | 
[####################] 100%  writing vcf file      | 0:00:00 | s7 | 
[####################] 100%  building arr

## Assembly stats
See the github page for stats of each assembly. 

In [11]:
## reoload assemblies from their JSON files
Can_min4 = ip.load_json("analysis-ipyrad/Canarium-min4.json")
Can_min10 = ip.load_json("analysis-ipyrad/Canarium-min10.json")
Can_min20 = ip.load_json("analysis-ipyrad/Canarium-min20.json")
Can_min30n = ip.load_json("analysis-ipyrad/Canarium-min30-nout.json")

loading Assembly: Canarium-min4
from saved path: ~/Documents/Canarium/analysis-ipyrad/Canarium-min4.json
loading Assembly: Canarium-min10
from saved path: ~/Documents/Canarium/analysis-ipyrad/Canarium-min10.json
loading Assembly: Canarium-min20
from saved path: ~/Documents/Canarium/analysis-ipyrad/Canarium-min20.json
loading Assembly: Canarium-min30-nout
from saved path: ~/Documents/Canarium/analysis-ipyrad/Canarium-min30-nout.json
